In [1]:
import os                       # for working with files
import numpy as np              # for numerical computationss
import pandas as pd             # for working with dataframes
import seaborn as sns
import matplotlib.pyplot as plt # for plotting informations on graph and images using tensors
from PIL import Image           # for checking images
import tensorflow as tf 
from  tensorflow import keras
import itertools
from sklearn.metrics import precision_score, accuracy_score, recall_score, confusion_matrix, ConfusionMatrixDisplay

%matplotlib inline

In [2]:
lung_dir = "../input/lung-and-colon-cancer-histopathological-images/lung_colon_image_set/lung_image_sets"
lungs = os.listdir(lung_dir)

In [3]:
lungs

['lung_aca', 'lung_scc', 'lung_n']

In [4]:
for i in lungs:
    print("%s : %d"%(i,len(os.listdir(lung_dir+"/"+i))))

lung_aca : 5000
lung_scc : 5000
lung_n : 5000


In [5]:
train_dataset = tf.keras.utils.image_dataset_from_directory(
    lung_dir,
    labels='inferred',
    label_mode='categorical',
    class_names=None,
    color_mode='rgb',
    batch_size=64,
    image_size=(256, 256),
    shuffle=True,
    seed=47,
    validation_split=0.2,
    subset="training"
)

val_dataset = tf.keras.utils.image_dataset_from_directory(
    lung_dir,
    labels='inferred',
    label_mode='categorical',
    class_names=None,
    color_mode='rgb',
    batch_size=64,
    image_size=(256, 256),
    shuffle=True,
    seed=47,
    validation_split=0.2,
    subset="validation"
)

Found 15000 files belonging to 3 classes.
Using 12000 files for training.


2022-11-24 08:09:20.117289: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-24 08:09:20.213447: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-24 08:09:20.214236: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-24 08:09:20.217332: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Found 15000 files belonging to 3 classes.
Using 3000 files for validation.


In [6]:
#optimization parameter setting.
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
val_dataset = val_dataset.prefetch(buffer_size=AUTOTUNE)

In [7]:
@tf.function
def squash(x, axis=-1):
    s_squared_norm = tf.math.reduce_sum(tf.math.square(x), axis, keepdims=True) + keras.backend.epsilon()
    scale = tf.math.sqrt(s_squared_norm) / (1 + s_squared_norm)
    return scale * x

@tf.function
def margin_loss(y_true, y_pred):
    lamb, margin = 0.5, 0.1
    return tf.math.reduce_sum((y_true * tf.math.square(tf.nn.relu(1 - margin - y_pred)) + lamb * (
        1 - y_true) * tf.math.square(tf.nn.relu(y_pred - margin))), axis=-1)

#@tf.function
def safe_norm(s, axis=-1, epsilon=1e-7, keep_dims=False):
        squared_norm = tf.reduce_sum(tf.square(s),axis=axis,keepdims=keep_dims)
        return tf.sqrt(squared_norm + epsilon)
    

In [8]:
class Capsule(keras.layers.Layer):
   

    def __init__(self,
                 num_capsule,
                 dim_capsule,
                 **kwargs):
        super(Capsule, self).__init__(**kwargs)
        self.caps_n = num_capsule
        self.caps_dim = dim_capsule

    def get_config(self):
        config = super().get_config().copy()
        config.update({
        'num_capsule':  self.caps_n,
        'dim_capsule' : self.caps_dim,    
        })
        return config

    def build(self, input_shape):

        self.W = self.add_weight(name='W',
                    shape=[1, input_shape[1], self.caps_n, self.caps_dim, input_shape[-1]],
                    dtype=tf.float32,
                    initializer='glorot_uniform',
                    trainable=True)
        
        self.R = self.add_weight(name='R',
                    shape=[1, input_shape[1], self.caps_n],
                    dtype=tf.float32,
                    initializer='glorot_uniform',
                    trainable=True)
        
        
    def call(self, input_tensor):
        batch_size = input_tensor.shape[0]
        n=input_tensor.shape[1]
        k=self.caps_n
        
        W_tiled = tf.tile(self.W, [batch_size, 1, 1, 1, 1])
        
        R_tiled = tf.tile(self.R,[batch_size,1,1])
        R_tiled = tf.nn.softmax(R_tiled,axis=1)

        caps_output_expanded = tf.expand_dims(input_tensor, -1) # converting last dim to a column vector.
        caps_output_tile = tf.expand_dims(caps_output_expanded, 2)
        caps_output_tiled = tf.tile(caps_output_tile, [1, 1, self.caps_n, 1, 1]) # replicating the input capsule vector for every output capsule.
        caps_predicted = tf.matmul(W_tiled, caps_output_tiled) # this is performing element wise tf.matmul() operation.       
        weighted_prediction=tf.multiply(caps_predicted,tf.reshape(R_tiled,[batch_size,n,k,1,1]))
        weighted_sum = tf.reduce_sum(weighted_prediction, axis=1, keepdims=True)
        v=squash(weighted_sum, axis=-2)
        v = tf.squeeze(v, axis=[1,4])
        return v

In [9]:
c1=tf.keras.layers.Conv2D(8,kernel_size=5,strides=2,padding='valid',activation='relu')
c2=tf.keras.layers.Conv2D(8,kernel_size=9,strides=2,padding='valid',activation='relu')
c3=tf.keras.layers.Conv2D(8,kernel_size=9,strides=2,padding='valid',activation='relu')
c4=tf.keras.layers.Conv2D(8,kernel_size=11,strides=1,padding='valid',activation='relu')
#dc1=tf.keras.layers.DepthwiseConv2D(kernel_size=9,strides=(1, 1),padding='valid',activation='relu')
last=Capsule(3,8)
bn1=tf.keras.layers.BatchNormalization()
bn2=tf.keras.layers.BatchNormalization()
bn3=tf.keras.layers.BatchNormalization()
bn4=tf.keras.layers.BatchNormalization()

In [10]:
X=keras.Input((256,256,3),batch_size=64)

In [11]:
c4(c3(c2(c1(X))))

<KerasTensor: shape=(64, 16, 16, 8) dtype=float32 (created by layer 'conv2d_3')>

In [12]:
model_input = keras.Input(shape=(256, 256, 3), batch_size=64)
x=c1(model_input)
x=bn1(x,training=True)
x=c2(x)
x=bn2(x,training=True)
x=c3(x)
x=bn3(x,training=True)
x=c4(x)
x=bn4(x,training=True)
#x=dc1(x)
x=tf.reshape(x,[-1,16*16,8])
x=last(x)
x=safe_norm(x, axis=2)
model_output = x

In [13]:
model = keras.Model(model_input, model_output)

In [14]:
adam = tf.keras.optimizers.Adam(learning_rate=0.0001) 

model.compile(loss=margin_loss, optimizer=adam, metrics=tf.keras.metrics.CategoricalAccuracy())
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(64, 256, 256, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (64, 126, 126, 8)         608       
_________________________________________________________________
batch_normalization (BatchNo (64, 126, 126, 8)         32        
_________________________________________________________________
conv2d_1 (Conv2D)            (64, 59, 59, 8)           5192      
_________________________________________________________________
batch_normalization_1 (Batch (64, 59, 59, 8)           32        
_________________________________________________________________
conv2d_2 (Conv2D)            (64, 26, 26, 8)           5192      
_________________________________________________________________
batch_normalization_2 (Batch (64, 26, 26, 8)           32    

In [15]:
"""customize training loop."""

# Instantiate an optimizer to train the model.
base_learning_rate = 0.0001
optimizer = tf.keras.optimizers.Adam(learning_rate=base_learning_rate)
# Instantiate a loss function.
loss_fn = margin_loss

# Prepare the metrics.
train_acc_metric = tf.keras.metrics.CategoricalAccuracy()
val_acc_metric = tf.keras.metrics.CategoricalAccuracy()

In [17]:
epochs = 30
for epoch in range(epochs):
    print("\nepoch {}/{}".format(epoch+1,epochs))
    pbar = keras.utils.Progbar(target=int(train_dataset.cardinality()))
    metrics = {}

    # Iterate over the batches of the dataset.
    for step, (x_batch_train, y_true) in enumerate(train_dataset):
        #y_true = tf.keras.utils.to_categorical(y_batch_train,num_classes=2)
        with tf.GradientTape() as tape:
            y_pred=model(x_batch_train) # $ better design needed.
            # y_pred is prob. dist.
            loss_value = loss_fn(y_true,y_pred) # loss computation
        grads = tape.gradient(loss_value, model.trainable_weights) # back prop
        optimizer.apply_gradients(zip(grads, model.trainable_weights)) # weight update

        # Update training metric.
        train_acc_metric.update_state(y_true, y_pred)
        metrics.update({'train_acc':train_acc_metric.result()})
        pbar.update(step+1, values=metrics.items(), finalize=False)


    # Run a validation loop at the end of each epoch.
    for x_batch_val, y_batch_val in val_dataset:
      #y_batch_val=tf.keras.utils.to_categorical(y_batch_val,num_classes=2)
      val_pred = model(x_batch_val) # $ better design needed
      # Update val metrics
      val_acc_metric.update_state(y_batch_val, val_pred)

    metrics.update({'val_acc':val_acc_metric.result()})
    
    pbar.update(step+1, values=metrics.items(), finalize=True)
    
    # Reset training & val metrics at the end of each epoch
    train_acc_metric.reset_states()
    val_acc_metric.reset_states()


epoch 1/30
188/188 [==============================] - 52s 269ms/step - train_acc: 0.8721 - val_acc: 0.8773

epoch 2/30
188/188 [==============================] - 51s 262ms/step - train_acc: 0.8746 - val_acc: 0.8660

epoch 3/30
188/188 [==============================] - 51s 261ms/step - train_acc: 0.8813 - val_acc: 0.8777

epoch 4/30
188/188 [==============================] - 49s 252ms/step - train_acc: 0.8820 - val_acc: 0.8867

epoch 5/30
188/188 [==============================] - 51s 263ms/step - train_acc: 0.8856 - val_acc: 0.8830

epoch 6/30
188/188 [==============================] - 51s 265ms/step - train_acc: 0.8905 - val_acc: 0.8887

epoch 7/30
188/188 [==============================] - 92s 483ms/step - train_acc: 0.8949 - val_acc: 0.8957

epoch 8/30
188/188 [==============================] - 51s 262ms/step - train_acc: 0.8991 - val_acc: 0.8923

epoch 9/30
188/188 [==============================] - 52s 269ms/step - train_acc: 0.8997 - val_acc: 0.9080

epoch 10/30
188/188 [======